# Codice per Summary

Carichiamo in memoria i dati utilizziando gli scripts implementati in `create_db.py`.

In [1]:
from create_database import MakeData
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as ply

import plotly.graph_objs as go
init_notebook_mode(connected=True)
from plotly import tools

from sqlalchemy import create_engine

from config import PLOTLY_APIKEY, PLOTLY_USERNAME

con = create_engine('sqlite:///IT_schools.db')

## Studenti: quanti sono e da dove vengono

In [15]:
query = ''' 
select regione, 
sum(alunni) as alumni,
sum(italian) as italians,
sum(non_italian) as non_italians,
sum(eu) as eu,
sum(non_eu) as non_eu,
round(sum(italian)*1.0/sum(alunni) ,3) as italians_perc,
round(sum(non_italian)*1.0/sum(alunni) ,3) as non_italians_perc,
round(sum(eu)*1.0/sum(non_italian) ,3)  as eu_perc,
round(sum(non_eu)*1.0/sum(non_italian) , 3) as non_eu_perc


from (
select anagrafica.*, studenti_aggr.*
from 
studenti_aggr
inner join 
anagrafica
on studenti_aggr.codice_scuola = anagrafica.codice_scuola)
group by regione
'''
df = pd.read_sql_query(query,con = con, index_col='regione')
df

,alumni,italians,non_italians,eu,non_eu,italians_perc,non_italians_perc,eu_perc,non_eu_perc
regione,,,,,,,,,
ABRUZZO,148059,137605,10454,3253,7201,0.929,0.071,0.311,0.689
BASILICATA,69345,67036,2309,1102,1207,0.967,0.033,0.477,0.523
CALABRIA,242694,231733,10961,4745,6216,0.955,0.045,0.433,0.567
CAMPANIA,819041,799048,19993,6041,13952,0.976,0.024,0.302,0.698
EMILIA ROMAGNA,508499,430626,77873,11193,66680,0.847,0.153,0.144,0.856
FRIULI-VENEZIA G.,131332,116387,14945,3673,11272,0.886,0.114,0.246,0.754
LAZIO,679031,615415,63616,27633,35983,0.906,0.094,0.434,0.566
LIGURIA,159385,140147,19238,2649,16589,0.879,0.121,0.138,0.862
LOMBARDIA,1149475,984519,164956,23008,141948,0.856,0.144,0.139,0.861


In [22]:
import utils 
import importlib
importlib.reload(utils)

metric = 'non_italians_perc'
mgp = utils.MakeGeoMap(df[metric], 'Greens')

title = 'Percentage of foreign students per region <br> <i>Percentuale di studenti stranieri per regione </i>'
ispercentage = True

data, layout = mgp.make_figure(title, ispercentage)
fig = dict(data=[data], layout=layout)
#ff = iplot(fig)

ply.sign_in(PLOTLY_USERNAME, PLOTLY_APIKEY)
ply.iplot(fig, world_readable = True, filename='Foreign-percentage')

In [ ]:
metric = 'non_italians_perc'
mgp = utils.MakeGeoMap(df[metric], 'Greens')
data1, layout1 = mgp.make_figure()

metric = 'italians_perc'
mgp = utils.MakeGeoMap(df[metric], 'Greens')
data2, layout2 = mgp.make_figure()

fig = tools.make_subplots(rows=1, cols=2)

fig.append_trace(data1, 1, 1)
fig.append_trace(data2, 1, 2)
fig['layout'].update([layout1, layout2])

                     
iplot(fig)

In [ ]:
df1 = df.sort_values('eu_perc')
df2 = df.sort_values('non_eu_perc')

data1 = go.Bar(x=df1.eu_perc ,
               y=df1.index,
               orientation = 'h',
               hoverinfo = 'x')

data2 = go.Bar(x=df2.non_eu_perc ,
               y=df2.index,
               orientation = 'h',
               hoverinfo = 'x')

fig = tools.make_subplots(rows=1, cols=2, subplot_titles=('Studenti provenienti da nazioni EU', 
                                                          'Studenti provenienti da nazioni non-EU'))

fig.append_trace(data1, 1, 1)
fig.append_trace(data2, 1, 2)

    
fig['layout'].update(height=600, width=900, 
                     xaxis=dict(tickformat = ',.0%', domain=[0.05, 0.4]),
                     xaxis2=dict(tickformat = ',.0%', domain=[0.6, 1]),
                     showlegend=False
                    )
                     
iplot(fig)

## Scuola: quante sono e dove sono?

In [ ]:
# Schools per city
comunui_grouped = active_schools.groupby('codice_comune').size()
comunui_grouped.name = 'schools'
demog_tmp = demog.set_index('codice_comune', inplace = False, drop = True)
schools_per_city = pd.concat([demog_tmp, comunui_grouped], axis = 1)

ss = schools_per_city.groupby('regione').sum()

data1 = go.Scatter(x=ss['schools'],
                   y=ss['popolazione (09/10/2011)'],
                   mode = 'markers',
                   hoverinfo='x+y')

iplot([data1])

In [ ]:
active_schools.groupby(['codice_comune', 'grado']).size()